# Librerias necesarias

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from traduccion_sql_ln import *
from parser_SQL import *
from embeddings import *


# Configuración inicial

## Configuración embeddins

In [2]:
modelPath = "thenlper/gte-large"

model_kwargs = {'device':'cuda'}

encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

c:\Users\PC\miniconda3\envs\evariste-env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\PC\miniconda3\envs\evariste-env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
db = cargar_embeddings('emmbeddings_db', 'indices_paises_wikis', embeddings)

## Configuración LLM

In [4]:
retriever = db.as_retriever()

ollama = Ollama(
    base_url='http://localhost:3030',
    model="llama2-uncensored"
)

# Preparación para hacer las preguntas

In [5]:
system_prompt=("You are a highly intelligent question answering bot. "
               "If I ask you a question that is rooted in truth, you will give you the answer. "
               "If I ask you a question that is nonsense, trickery, or has no clear answer, you will respond with 'Unknown'. "
               "You will answer concisely. "
               "Use the given context as a support to answer the question if you can't answer the question."
               "Context: {context}")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Answer the query.\n{format_instructions}\n{question}\n"),
    ],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def crear_instrucciones(columnas: list[str]):
    texto = "Format the information as a table with columns for "
    if len(columnas) > 0:
        texto += columnas.pop(0)
    
    for _ in range(len(columnas) -1):
        texto += f", {columnas.pop(0)}"
    
    texto += f"and {columnas.pop(0)}."

    texto += " Your response should be a table"
    
    return (lambda *args: texto)

## Haciendo consultas

In [24]:
consulta_sql = '''SELECT T2.Language FROM country AS T1 JOIN countrylanguage AS T2 ON T1.Code = T2.CountryCode WHERE T1.HeadOfState = "Beatrix" AND T2.IsOfficial = "T"'''
    
lista_miniconsulta = obtener_miniconsultas(consulta_sql)['independientes']

for miniconsulta in lista_miniconsulta:
    traduccion = traducir_miniconsulta_sql(miniconsulta)
    proyecciones = traducir_proyecciones(miniconsulta)
    lista_columnas_condiciones = obtener_columnas_condicion(miniconsulta)

    
    rag_chain = (
        {"context": retriever | format_docs, 
        "question": RunnablePassthrough(),
        "format_instructions": crear_instrucciones(proyecciones + lista_columnas_condiciones)}
        | prompt
        | ollama
        | StrOutputParser()
    )

    print("################################################")
    print(f"Pregunta: {traduccion}")
    print("Respuesta: ")
    print(rag_chain.invoke(traduccion))

    

################################################
Pregunta: Give me the Code of the country where HeadOfState is equal to "Beatrix"
Respuesta: 
| Country | HeadOfState |
| --- | --- |
| Netherlands | Beatrix |
